<a href="https://colab.research.google.com/github/JoannaLe/cancer-detection/blob/master/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential, Model

import numpy as np
import os
import matplotlib.pyplot as plt

GOOGLE_APPLICATION_CREDENTIALS = '/content/cancer-detection-fa915e095a9d.json'

In [3]:
# Open Google Cloud Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
print(os.getcwd())
os.chdir("/content")
!ls

/content
gdrive	sample_data


In [5]:
# get X training data
x_path = "gdrive/My Drive/cancer-detection/patch-zoom-6"
print(os.listdir(x_path))
X_train = []

for patch_file in os.listdir(x_path):
  patch_path = os.path.join(x_path, patch_file)
  patch_arr = np.load(patch_path)
  X_train.append(patch_arr)
  
print(type(X_train[0]))
print(X_train[0])
print(len(X_train))
X_train = np.array(X_train)

# get Y training data
y_path = "gdrive/My Drive/cancer-detection/mask-zoom-6"
print(os.listdir(y_path))
y_train = []

for mask_file in os.listdir(y_path):
  mask_path = os.path.join(y_path, mask_file)
  mask = open(mask_path, 'r').read()
  y_train.append(int(mask))
  
print(type(y_train[0]))
print(y_train)
print(len(y_train))
y_train = np.array(y_train)

['091-patch-0.npy', '091-patch-1.npy', '091-patch-2.npy', '091-patch-3.npy', '091-patch-4.npy', '091-patch-5.npy', '091-patch-6.npy', '091-patch-7.npy', '091-patch-8.npy', '091-patch-9.npy', '091-patch-10.npy', '091-patch-11.npy', '091-patch-12.npy', '091-patch-13.npy', '091-patch-14.npy', '091-patch-15.npy', '091-patch-16.npy', '091-patch-17.npy', '091-patch-18.npy', '091-patch-19.npy', '091-patch-20.npy', '091-patch-21.npy', '091-patch-22.npy', '091-patch-23.npy', '091-patch-24.npy', '091-patch-25.npy', '091-patch-26.npy', '091-patch-27.npy', '091-patch-28.npy', '091-patch-29.npy', '091-patch-30.npy', '091-patch-31.npy', '091-patch-32.npy', '091-patch-33.npy', '091-patch-44.npy', '091-patch-43.npy', '091-patch-34.npy', '091-patch-35.npy', '091-patch-45.npy', '091-patch-36.npy', '091-patch-39.npy', '091-patch-37.npy', '091-patch-47.npy', '091-patch-46.npy', '091-patch-38.npy', '091-patch-41.npy', '091-patch-48.npy', '091-patch-40.npy', '091-patch-49.npy', '091-patch-50.npy', '091-patc

In [0]:
# test 1 model at zoom level 6
img_shape = (224,224,3)
EPOCHS = 10
BATCH_SIZE = 32

model1 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
model1.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model1.summary()

In [0]:
model1.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

In [6]:
img_shape = (224,224,3)
EPOCHS = 10
BATCH_SIZE = 32

model = Sequential()
model.add(Conv2D(64, (3,3), activation='relu', input_shape=img_shape))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
flatten_1 (Flatten)          (None, 3154176)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                201867328 
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 201,869,185
Trainable params: 201,869,185
Non-trainable params: 0
_________________________________________________________________


In [7]:
print(X_train.shape)

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

(72, 224, 224, 3)
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
72/72 [==============================] - 12s 170ms/step - loss: 3.9834 - acc: 0.6528
Epoch 2/10
72/72 [==============================] - 2s 30ms/step - loss: 0.4477 - acc: 0.9722
Epoch 3/10
72/72 [==============================] - 2s 30ms/step - loss: 0.4477 - acc: 0.9722
Epoch 4/10
72/72 [==============================] - 2s 31ms/step - loss: 0.4477 - acc: 0.9722
Epoch 5/10
72/72 [==============================] - 2s 31ms/step - loss: 0.4477 - acc: 0.9722
Epoch 6/10
72/72 [==============================] - 2s 31ms/step - loss: 0.4477 - acc: 0.9722
Epoch 7/10
72/72 [==============================] - 2s 31ms/step - loss: 0.4477 - acc: 0.9722
Epoch 8/10
72/72 [==============================] - 2s 31ms/step - loss: 0.4477 - acc: 0.9722
Epoch 9/10
72/72 [==============================] - 2s 31ms/step - loss: 0.4477 - acc: 0.9722
Epoch 10/10
72/72 [==============================] - 2s 31ms/step - loss: 0.4477 - acc

## Model 1
4 resnet models with a concatenation layer

In [0]:
img_shape = (300,300,3)

model1 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
model2 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
model3 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
model4 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)

# concatenation layer
concat = Average()([model1.output, model2.output, model3.output, model4.output])

full_model = Model([model1.input, model2.input, model3.input, model4.input], concat)


In [0]:
# to train, call
EPOCHS = 20
BATCH_SIZE = 32

full_model.fit([X_train1, X_train2, X_train3, X_train4],
               [Y_train1, Y_train2, Y_train3, Y_train4],
                epochs=EPOCHS,
                batch_size=BATCH_SIZE)

score = model.evaluate([X_test1, X_test2, X_test3, X_test4],
                       [Y_test1, Y_test2, Y_test3, Y_test4],
                       batch_size=BATCH_SIZE)

# Model 2
Transfer Learning using weights from Resnet \\
Extract features from Resnet weights and training/test data

In [0]:
img_shape = (300,300,3)

base = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
